In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

In [15]:
#경로 설정
directory= 'C:\\Users\\USER\\PycharmProjects\\ERC\\'
os.chdir(directory)

annotation 취합
---

In [16]:
ann = os.path.join(directory, 'data/20/annotation')
files = os.listdir(ann)
filescnt_wav = [i for i in files if i.endswith('.csv')]

df_all = pd.DataFrame()
for i in range(0, len(filescnt_wav)) :
    file = filescnt_wav[i]
    df = pd.read_csv(os.path.join(ann, file), encoding = 'utf-8')
    df['filename'] = file.split('_eval')[0]
    df_all = pd.concat([df_all, df])
sess_df = df_all.drop([0],axis = 0)

wav/txt 취합
---

In [17]:
def search(dirname):
    result = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        full_filename = dirname + '//' + filename
        result.append(full_filename)    
    return result

In [18]:
wav_path = os.path.join(directory, 'data/20/wav')
wav_sess = search(wav_path)

In [19]:
txt_files = []
wav_files = []

for i in wav_sess:
    filenames = os.listdir(i)
    direc = i[42:]
    for filename in filenames:
        full_filename = direc + '//' + filename
        if full_filename[-3:] == 'wav':
            wav_files.append(full_filename)
        elif full_filename[-3:] == 'txt':
            txt_files.append(full_filename)
        else:
            print('wrong!')

In [20]:
total = pd.DataFrame({'Audio': wav_files,
                      'Text': txt_files,
                      'Emotion': wav_files,
                      'Arousal': wav_files,
                      'Valence': wav_files})

전체 데이터 취합
---

In [21]:
path = directory+'data/20/'
for i in tqdm(range(total.shape[0])):
    directory = os.path.join(path, total.iloc[i, 1])
    with open (directory, 'r') as text:
        string = text.readlines()
    total.iloc[i, 1] = string[0].replace('\n', '')

100%|██████████████████████████████████████████████████████████████████████████| 13462/13462 [00:02<00:00, 4773.53it/s]


In [23]:
for i in tqdm(range(total.shape[0])):
    file_name = total.iloc[i, 2][16:-4]
    total.iloc[i, 2] = sess_df[sess_df['Segment ID'] == file_name].iloc[0, 4]
    total.iloc[i, 3] = sess_df[sess_df['Segment ID'] == file_name].iloc[0, 6]
    total.iloc[i, 4] = sess_df[sess_df['Segment ID'] == file_name].iloc[0, 5]

100%|███████████████████████████████████████████████████████████████████████████| 13462/13462 [00:34<00:00, 395.69it/s]


In [24]:
total

,Audio,Text,Emotion,Arousal,Valence
0,wav//Session01//Sess01_script01_User001F_001.wav,n/ 아 친구들도? l/,neutral,2.8,3.8
1,wav//Session01//Sess01_script01_User001F_002.wav,l/ 나는 생일?,neutral,3.1,3.3
2,wav//Session01//Sess01_script01_User001F_003.wav,생일날이면은 b/ 내가 고기를 되게 좋아하니까 엄마도 그걸 아니까 미역국도 같이 해...,neutral,3.1,3.3
3,wav//Session01//Sess01_script01_User001F_004.wav,b/ 선물이라 이 보통 돈으로 주시지. l/,happy,3.1,3.6
4,wav//Session01//Sess01_script01_User001F_005.wav,l/ 용돈으로 주시고.,neutral,3.2,3.5
...,...,...,...,...,...
13457,wav//Session40//Sess40_script06_User080F_021.wav,"어, 암튼 그런 것처럼 이게 그니까 여자들은 근데 그런 거에 대해서 어 나쁘게 생각 어.",neutral,3.5,2.6
13458,wav//Session40//Sess40_script06_User080F_022.wav,"그치, 근데 그거를 u/ 아 모르겠어.",neutral,3.7,3
13459,wav//Session40//Sess40_script06_User080F_023.wav,"u/ 그걸 점점 발전을 시키잖아, 사람처럼 느껴지게.",neutral,3.7,2.9
13460,wav//Session40//Sess40_script06_User080F_024.wav,그래서 가격도 달라지고 어 그니까 그런 식으로 이렇게 잘못 사용하게 되면 그니까 잘...,neutral,3.7,2.4


In [40]:
for i in range(total.shape[0]):
    text = total.iloc[i, 1]
    text = text.replace('c/','').replace('n/','').replace('N/','').replace('u/','').replace('l/','')
    text = text.replace('b/','').replace('*','').replace('+','').replace(',','')
    total.iloc[i, 1] = text

In [42]:
list_id = []
list_sex = []
for i in range(total.shape[0]):
    list_id.append(total.iloc[i, 0][16:-4])
    list_sex.append(total.iloc[i, 0][39])
    
total['id'] = list_id
total['Sex'] = list_sex

In [43]:
total = total[['id', 'Sex', 'Emotion', 'Arousal', 'Valence']]
total.to_csv('C:\\Users\\USER\\PycharmProjects\\ERC\\data\\total.csv', index=False, encoding="utf-8-sig") 
total.head()

,id,Sex,Emotion,Arousal,Valence
0,Sess01_script01_User001F_001,F,neutral,2.8,3.8
1,Sess01_script01_User001F_002,F,neutral,3.1,3.3
2,Sess01_script01_User001F_003,F,neutral,3.1,3.3
3,Sess01_script01_User001F_004,F,happy,3.1,3.6
4,Sess01_script01_User001F_005,F,neutral,3.2,3.5


불러오기
---

In [81]:
sig_n = 'ibi' #eda, ibi, temp

signal = pd.read_csv('data/t_{}.csv'.format(sig_n))
total = pd.read_csv('data/total.csv')

In [82]:
list_emotion = ['neutral', 'happy', 'surprise', 'angry', 'sad', 'disqust', 'fear']
condition = total['Emotion'].isin(list_emotion)
total_new = total[condition]

In [83]:
#클래스 균형 맞추기
df_valance = pd.DataFrame()
for emo in list_emotion:
    data_emo = total_new[total_new['Emotion'] == emo]
    data_ds = resample(data_emo, n_samples=150, random_state=42)
    df_valance = pd.concat([df_valance, data_ds])

In [87]:
total_new[['Emotion']].value_counts()
#df_valance[['Emotion']].value_counts()

Emotion 
neutral     11120
happy        1183
surprise      156
angry         144
sad           121
disqust        61
fear           43
dtype: int64

In [88]:
all_eda = df_valance.merge(signal, how='left', on='id')
#all_eda = total_new.merge(signal, how='left', on='id')

all_eda = all_eda.fillna(0)
all_eda.to_csv('data/{}.csv'.format(sig_n), index=False)

데이터 조합
---

In [102]:
sig_n = 'temp' #eda, ibi, temp
sig_n2 = 'ibi'

signal = pd.read_csv('data/{}.csv'.format(sig_n))
signal2 = pd.read_csv('data/{}.csv'.format(sig_n2))

signal_new = signal.merge(signal2, how='inner', on=['id', 'Sex', 'Emotion', 'Arousal', 'Valence'] )
signal_new.to_csv('data/{}_{}.csv'.format(sig_n, sig_n2), index=False)